In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')
import pyNetLogo

path=r'C:\Program Files\NetLogo 6.1.0'
netlogo = pyNetLogo.NetLogoLink(gui=True, thd=False, netlogo_home=path, netlogo_version='6.1')

model=r'C:\Users\Kevin\OneDrive - University College Dublin\Desktop\fyp\FYP_Model_Final.nlogo'
netlogo.load_model(model)

Please change your settings in the netlogo interface to your desired settings before running the next cell.

In [ ]:
closeContactsList = []
xyListRecovered = []
xyListVirus = []
xyListNormal = []
ticks = 0

netlogo.command('setup')
numAgents = netlogo.report('count turtles')
#Code from pyNetLogo Documentation + https://stackoverflow.com/questions/53236798/python-how-to-store-values-from-a-while-loop-into-a-pandas-dataframe
while netlogo.report('all? turtles [ not virus? ]') != True: 
    netlogo.command('Go')
    netlogo.command('make-list')
    infected = netlogo.report('count turtles with [ virus? ]')
    recovered = netlogo.report('count turtles with [ recovered? ]')
    closeContacts = netlogo.report('count links')
    avgCloseContacts = netlogo.report('(count links / 2) / NumAgents')
    normal = netlogo.report('count turtles with [ not virus? and not recovered?]')
    data = {
        'Infected':infected,
        'Recovered':recovered,
        'Close Contacts' :closeContacts,
        'Avg Close Contacts' :avgCloseContacts,
        'Healthy' :normal,
        'Ticks':ticks
    }
    closeContactsList.append(data)
    if infected > 0:
        listVirus = netlogo.report('xylist-virus')
        coordsVirus = {
            'xylist-virus' :listVirus,
            'Ticks':ticks
        }
        xyListVirus.append(coordsVirus)
    if recovered > 0:
        listRecovered = netlogo.report('xylist-recovered')
        coordsRecovered = {
         'xylist-recovered' :listRecovered,
         'Ticks' :ticks
        }
        xyListRecovered.append(coordsRecovered)
    if recovered + infected != numAgents:
        listNormal = netlogo.report('xylist-normal')
        coordsNormal = {
            'xylist-normal' :listNormal,
            'Ticks':ticks
        }
        xyListNormal.append(coordsNormal)
    ticks +=1
    
df = pd.DataFrame(closeContactsList)
df3 = pd.DataFrame(xyListRecovered)
df4 = pd.DataFrame(xyListVirus)
df5 = pd.DataFrame(xyListNormal)

#explode array to allow each row to be a new x y point in dataframe for both recovered and infected agents
df_recovered_cords = df3.explode("xylist-recovered")
df_virus_cords = df4.explode("xylist-virus")
df_normal_cords = df5.explode("xylist-normal")

#Create lists to store X and Y Value and seperate them from same column
x_recovered_list = []
y_recovered_list = []
id_recovered_list = []
x_virus_list = []
y_virus_list = []
id_virus_list = []
x_normal_list = []
y_normal_list = []
id_normal_list = []

#for loop to append x and y to seperate lists to be seperate columns
for i in df_recovered_cords["xylist-recovered"]:
    x_recovered_list.append(i[0])
    y_recovered_list.append(i[1])
    id_recovered_list.append(i[2])

for i in df_virus_cords["xylist-virus"]:
    x_virus_list.append(i[0])
    y_virus_list.append(i[1])
    id_virus_list.append(i[2])
    
for i in df_normal_cords["xylist-normal"]:
    x_normal_list.append(i[0])
    y_normal_list.append(i[1])
    id_normal_list.append(i[2])
    
#add seperated data back to dataframe as seperate columns
df_recovered_cords['x'] = x_recovered_list
df_recovered_cords['y'] = y_recovered_list
df_recovered_cords['id'] = id_recovered_list

df_virus_cords['x'] = x_virus_list
df_virus_cords['y'] = y_virus_list
df_virus_cords['id'] = id_virus_list

df_normal_cords['x'] = x_normal_list
df_normal_cords['y'] = y_normal_list
df_normal_cords['id'] = id_normal_list
                          
#Drop unnecessary rows (ticks, old xylist from netlogo)
df_virus_cords = df_virus_cords.drop(['xylist-virus'], axis = 1)
df_recovered_cords = df_recovered_cords.drop(['xylist-recovered'], axis = 1)
df_normal_cords = df_normal_cords.drop(['xylist-normal'], axis = 1)

#add Column to each dataset with default value of 'infected', 'recovered' or 'normal' to be used as 'color' in graph
df_virus_cords['status'] = 'virus'
df_recovered_cords['status'] = 'recovered'
df_normal_cords['status'] = 'healthy'

#merge/join all dataframes together so they can all be plotted together
df_simulation = pd.concat([df_normal_cords,df_virus_cords,df_recovered_cords], ignore_index=True)

for i in range(ticks):
    dummy_normal = {'Ticks': i, 'x':18, 'y':18, 'id':1001.0, 'status':'healthy'}
    dummy_recovered = {'Ticks': i, 'x':18, 'y':18, 'id':1002.0, 'status':'recovered'}
    dummy_virus = {'Ticks': i, 'x':18, 'y':18, 'id':1003.0, 'status':'virus'}
    df_simulation = df_simulation.append(dummy_normal, ignore_index=True)
    df_simulation = df_simulation.append(dummy_recovered, ignore_index=True)
    df_simulation = df_simulation.append(dummy_virus, ignore_index=True)
    
#Save data to csv 
df.to_csv(r'C:\Users\Kevin\OneDrive - University College Dublin\Desktop\fyp\Dash Project\data\contacts_dataframe.csv', index = False, header=True)
df_simulation.to_csv(r'C:\Users\Kevin\OneDrive - University College Dublin\Desktop\fyp\Dash Project\data\agents_dataframe.csv', index = False, header=True)

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

# visit http://127.0.0.1:8050/ in your web browser.
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Load data from csv included in dash file format.
df = pd.read_csv(
    "Dash Project/data/contacts_dataframe.csv"
)
df_simulation = pd.read_csv(
    "Dash Project/data/agents_dataframe.csv"
)

#animations, app, external css and color.
animations = {
    'Scatter': px.scatter(
        df_simulation, x="x", y="y", animation_frame="Ticks", animation_group="id",
        hover_name="id", color = 'status', width = 800, height = 800,
        range_x=[-17,17], range_y=[-17,17]),
    'Close Contacts': px.line(
        df, x="Ticks", y = "Close Contacts", hover_name="Close Contacts", width = 800, height = 800,
    )
}

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

#app layout
app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.RadioItems(
                id='selection',
                options=[{'label': x, 'value': x} for x in animations],
                value='Scatter'
            ),
            dcc.Graph(id="graph")
        ],
        style={'width': '49%', 'display': 'inline-block', 'float': 'center', 'padding-left' : '33%'})
        
                  
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),
    html.Div([
        html.Div([
                dcc.Graph(
                    id='Model2-Graph-CloseContacts',
                    figure={
                        'data': [
                            
                            {'x': df['Ticks'], 'y': df['Avg Close Contacts'], 'mode' : 'line+markers', 'type': 'scatter', 'name': 'Avg Close Contacts'},
                            
                        ],
                        'layout': {
                            'title': 'Average Close Contacts',
                            'xaxis': {'title': 'Time in Ticks'},
                            'yaxis': {'title': 'Number of Contacts'},
                            'width' : '50%',
                            'height' : '50%',
                        }
                    },
                ),
        ], style={'width': '25%', 'float': 'left', 'display': 'inline-block', 'padding-right': '5%'}),
        html.Div([
                dcc.Graph(
                    id='Model2-Graph-IRH',
                    figure={
                        'data': [
                            {'x': df['Ticks'], 'y': df['Infected'], 'type': 'line', 'name': 'Infected', 'line':dict(color='red')},
                            {'x': df['Ticks'], 'y': df['Recovered'], 'type': 'line', 'name': 'Recovered', 'line':dict(color='green')},
                            {'x': df['Ticks'], 'y': df['Healthy'], 'type': 'line', 'name': 'Healthy', 'line':dict(color='blue')},
                        ],
                        'layout': {
                            'title': 'Healthy, Infected and Recovered',
                            'xaxis': {'title': 'Time in Ticks'},
                            'yaxis': {'title': 'Number of Agents'},
                            'width' : '50%',
                            'height' : '50%',
                        }
                    }
                )
        ], style={'width': '25%', 'float': 'right', 'display': 'inline-block', 'padding-left': '5%'}),
    ], style={'width': '49%', 'float': 'center', 'display': 'inline-block', 'padding-left': '15%'})
])

@app.callback(
    Output("graph", "figure"), 
    [Input("selection", "value")]
)
def display_animated_graph(s):
    return animations[s]

app.run_server(debug=True, use_reloader=False)

In [ ]:
#Close program
netlogo.kill_workspace()